In [ ]:
!pip install transformers
!pip install datasets --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.0 MB/s 
     |████████████████████████████████| 6.6 MB 44.5 MB/s 
     |████████████████████████████████| 120 kB 45.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 5.5 MB/s 
     |████████████████████████████████| 212 kB 54.2 MB/s 
     |████████████████████████████████| 115 kB 54.6 MB/s 
     |████████████████████████████████| 127 kB 57.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from transformers import TFAutoModelForMaskedLM

model_checkpoint = "distilbert-base-uncased"
model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForMaskedLM: ['activation_13']
- This IS expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertForMaskedLM were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForMaskedLM for predictions without further training.


In [ ]:
model(model.dummy_inputs)  # Build the model
model.summary()

Model: "tf_distil_bert_for_masked_lm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 vocab_transform (Dense)     multiple                  590592    
                                                                 
 vocab_layer_norm (LayerNorm  multiple                 1536      
 alization)                                                      
                                                                 
 vocab_projector (TFDistilBe  multiple                 23866170  
 rtLMHead)                                                       
                                                                 
Total params: 66,985,530
Trainable params: 66,985,530
Non-trainable params: 0
__________________________

In [ ]:
text = "This is a great [MASK]."


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
import numpy as np
import tensorflow as tf

inputs = tokenizer(text, return_tensors="np")
token_logits = model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = np.argwhere(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Pick the [MASK] candidates with the highest logits
# We negate the array before argsort to get the largest, not the smallest, logits
top_5_tokens = np.argsort(-mask_token_logits)[:5].tolist()

for token in top_5_tokens:
    print(f">>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}")

>>> This is a great deal.
>>> This is a great success.
>>> This is a great adventure.
>>> This is a great idea.
>>> This is a great feat.


In [ ]:
# LOAD DATASET
# import pandas as pd
# df = pd.read_parquet('all_2020_US_Election_News.parquet', engine='pyarrow')

In [ ]:
# MOUNT DRIVE
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd
%cd /content/drive/My Drive/Independent Study/
df = pd.read_parquet('all_2020_US_Election_News.parquet', engine='pyarrow')

# LOADING DATASET IN HUGGING FACE FORM
# from datasets import load_dataset
# df = load_dataset("parquet", data_files={'all_2020_US_Election_News.parquet'})

/content/drive/My Drive/Independent Study


In [ ]:
df

,Unnamed: 0,id,author,date,title,publication,text,keywords,summary
0,0,1,The Associated Press,2020-03-23 19:19:00,Review: 'The Glass Hotel a tragic tale for the...,ABC News,"The Glass Hotel, Alfred A. Knopf, by Emily St....","['hotel', 'tragic', 'vancouver', 'end', 'revie...","The Glass Hotel, Alfred A. Knopf, by Emily St...."
1,1,2,COLLEEN LONG and MARIA VERZA Associated Press,2020-03-23 11:21:34,Malaria drugs' promise for coronavirus spurs h...,ABC News,Malaria drugs' promise for coronavirus spurs h...,"['drug', 'virus', 'spurs', 'malaria', 'health'...",Malaria drugs' promise for coronavirus spurs h...
2,2,3,Ben Gittleson,2020-03-23 18:37:34,Malaria drugs' promise for coronavirus spurs h...,ABC News,Malaria drugs' promise for coronavirus spurs h...,"['drug', 'virus', 'spurs', 'malaria', 'health'...",Malaria drugs' promise for coronavirus spurs h...
3,3,4,Libby Cathey,2020-03-23 18:37:37,Senate showdown over pandemic relief stalls as...,ABC News,Senate showdown over pandemic relief stalls as...,"['relief', 'democrats', 'pelosi', 'vote', 'lea...",Senate showdown over pandemic relief stalls as...
4,4,5,ABC News,2020-03-23 19:52:06,Fallout of '08 bailout looms over Washington n...,ABC News,Fallout of '08 bailout looms over Washington n...,"['financial', 'money', '08', 'street', 'wall',...",On the table: a nearly $2 trillion rescue pack...
...,...,...,...,...,...,...,...,...,...
144367,144367,144714,"USA TODAY, Aimee Picchi, Special to USA TODAY",2021-01-21 08:31:10,'Andy spread election fraud lies': Arizona Rep...,USA Today,U.S. Rep. Andy Biggs two brothers have publicl...,"['election', 'lies', 'spread', 'brothers', 'me...",U.S. Rep. Andy Biggs two brothers have publicl...
144368,144368,144715,"Reviewed, Rachel Dube, Reviewed.com",2021-01-21 18:59:35,Amazon offers to help Biden administration wit...,USA Today,Amazon is offering its help to President Joe B...,"['vaccine', 'administration', 'help', 'letter'...",Amazon is offering its help to President Joe B...
144369,144369,144716,"Reviewed, Courtney Campbell, Reviewed",2021-01-21 20:35:05,Taron Johnson got out of the doghouse when Buf...,USA Today,The last time the Buffalo Bills played the Kan...,"['bills', 'didnt', 'play', 'played', 'buffalo'...",The last time the Buffalo Bills played the Kan...
144370,144370,144717,"USA TODAY, Charisse Jones and Matt Wynn, USA T...",2021-01-21 05:01:02,President Biden and Vice President Harris visi...,USA Today,President Biden and Vice President Harris visi...,"['tomb', 'vice', 'unknown', 'wreathlaying', 'c...",President Biden and Vice President Harris visi...


In [ ]:
# for index, row in df.iterrows():
#   if row.publication == 'ABC News':
#     row['leaning'] = 'L'
#   break
def label_leaning(row):
  if row['publication'] == 'ABC News' or row['publication'] == 'CBS News' or row['publication'] == 'CNN' or row['publication'] == 'MSNBC' or row['publication'] == 'The Washington Post' or row['publication'] == 'Al Jazeera English' or row['publication'] == 'Google News' or row['publication'] == 'Buzzfeed':
    return 'L'
  elif row['publication'] in ['Fox News', 'Breitbart News', 'The Washington Times']:
    return 'R'
  return 'C'

df['leaning'] = df.apply (lambda row: label_leaning(row), axis=1)


In [ ]:
df.iloc[0]

Unnamed: 0                                                     0
id                                                             1
author                                      The Associated Press
date                                         2020-03-23 19:19:00
title          Review: 'The Glass Hotel a tragic tale for the...
publication                                             ABC News
text           The Glass Hotel, Alfred A. Knopf, by Emily St....
keywords       ['hotel', 'tragic', 'vancouver', 'end', 'revie...
summary        The Glass Hotel, Alfred A. Knopf, by Emily St....
leaning                                                        L
Name: 0, dtype: object

In [ ]:
print(df.size)

1443720


In [ ]:
df2 = pd.DataFrame().assign(text=df['text'], leaning=df['leaning'])
df2 = df2.dropna()

df2.size

277754

In [ ]:
grouped_df = df2.groupby('leaning')

In [ ]:
grouped_df = df2.groupby('leaning').apply(lambda a: a[:])

In [ ]:
dfs = [x for _, x in df2.groupby('leaning')]

In [ ]:
C_dataset, L_dataset, R_dataset = dfs[0], dfs[1], dfs[2]

In [ ]:
C_dataset.to_csv (r'export_C_dataframe.csv', index = False, header=True)
L_dataset.to_csv (r'export_L_dataframe.csv', index = False, header=True)
R_dataset.to_csv (r'export_R_dataframe.csv', index = False, header=True)

In [ ]:
# LOADING L DATASET IN HUGGING FACE FORM
from datasets import load_dataset
%cd /content/drive/My Drive/Independent Study/

dataset = load_dataset("csv", data_files={'export_C_dataframe.csv'})

/content/drive/My Drive/Independent Study


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-16cea90c698ed0c2/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset = dataset['train']

In [ ]:
train = dataset.shuffle(seed=42).select(range(25000))
test = dataset.shuffle(seed=42).select(range(25000, 50000))
unsupervised = dataset.shuffle(seed=42).select(range(50000, len(dataset)))

In [ ]:
# pip install datasets
my_dataset_dict = {
    "train": train,
    "test": test,
    "unsupervised": unsupervised
    }

In [ ]:
len(my_dataset_dict['unsupervised'])

14752

In [ ]:
my_dataset_dict

{'train': Dataset({
     features: ['text', 'leaning'],
     num_rows: 25000
 }), 'test': Dataset({
     features: ['text', 'leaning'],
     num_rows: 25000
 }), 'unsupervised': Dataset({
     features: ['text', 'leaning'],
     num_rows: 14752
 })}

In [ ]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

# my_dataset_dict = my_dataset_dict.dropna()
# my_dataset_dict = my_dataset_dict['train']['tr'].tolist()
# texts_encodings = tokenizer(texts, truncation=True, padding=True)

# Use batched=True to activate fast multithreading!
train_tokenized_datasets = my_dataset_dict['train'].map(
    tokenize_function, batched=True, remove_columns=["text", "leaning"]
)
test_tokenized_datasets = my_dataset_dict['test'].map(
    tokenize_function, batched=True, remove_columns=["text", "leaning"]
)
unsupervised_tokenized_datasets = my_dataset_dict['unsupervised'].map(
    tokenize_function, batched=True, remove_columns=["text", "leaning"]
)

  0%|          | 0/25 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (669 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/15 [00:00<?, ?ba/s]

In [ ]:
print(len(train_tokenized_datasets), len(test_tokenized_datasets), len(unsupervised_tokenized_datasets))

25000 25000 14752


In [ ]:
tokenized_dataset = {
     "train": train_tokenized_datasets,
    "test": test_tokenized_datasets,
    "unsupervised": unsupervised_tokenized_datasets
}

In [ ]:
tokenized_dataset

{'train': Dataset({
     features: ['input_ids', 'attention_mask', 'word_ids'],
     num_rows: 25000
 }), 'test': Dataset({
     features: ['input_ids', 'attention_mask', 'word_ids'],
     num_rows: 25000
 }), 'unsupervised': Dataset({
     features: ['input_ids', 'attention_mask', 'word_ids'],
     num_rows: 14752
 })}

In [ ]:
tokenizer.model_max_length
chunk_size = 128


In [ ]:
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_dataset["train"][:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Review {idx} length: {len(sample)}'")

'>>> Review 0 length: 17'
'>>> Review 1 length: 53'
'>>> Review 2 length: 669'


In [ ]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated reviews length: {total_length}'")

'>>> Concatenated reviews length: 739'


In [ ]:
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 99'


In [ ]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
tokenized_dataset

{'train': Dataset({
     features: ['input_ids', 'attention_mask', 'word_ids'],
     num_rows: 25000
 }), 'test': Dataset({
     features: ['input_ids', 'attention_mask', 'word_ids'],
     num_rows: 25000
 }), 'unsupervised': Dataset({
     features: ['input_ids', 'attention_mask', 'word_ids'],
     num_rows: 14752
 })}

In [ ]:
lm_train_datasets = tokenized_dataset['train'].map(group_texts, batched=True)
lm_test_datasets = tokenized_dataset['test'].map(group_texts, batched=True)
lm_unsupervised_datasets = tokenized_dataset['unsupervised'].map(group_texts, batched=True)

lm_datasets = {
     "train": lm_train_datasets,
    "test": lm_test_datasets,
    "unsupervised": lm_unsupervised_datasets
}

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/15 [00:00<?, ?ba/s]

In [ ]:
lm_datasets

{'train': Dataset({
     features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
     num_rows: 179160
 }), 'test': Dataset({
     features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
     num_rows: 177465
 }), 'unsupervised': Dataset({
     features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
     num_rows: 104802
 })}

In [ ]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])


'with nuclear weapons. in tweets in farsi and english, irans ambassador and permanent representative to the un, kazem gharibabadi, said scapegoating and fearmongering are two common and classic methods used by demagogues. if you want to pursue a nuclear weapon programme, or you are seeking for an excuse to justify your lack of cooperation with the iaea or your outdated safeguard system, at least have the courage to admit it and pay the price for it, he said in reference to the international atomic energy agency. dont blame your wrongdoings on others by lies. the iranian'

In [ ]:
tokenizer.decode(lm_datasets["train"][1]["labels"])


'with nuclear weapons. in tweets in farsi and english, irans ambassador and permanent representative to the un, kazem gharibabadi, said scapegoating and fearmongering are two common and classic methods used by demagogues. if you want to pursue a nuclear weapon programme, or you are seeking for an excuse to justify your lack of cooperation with the iaea or your outdated safeguard system, at least have the courage to admit it and pay the price for it, he said in reference to the international atomic energy agency. dont blame your wrongdoings on others by lies. the iranian'

# **Fine-tuning DistilBERT with the Trainer API**


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
samples = [lm_datasets["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] [MASK] [MASK] the [MASK] to tailored : why [MASK]'s not [MASK] for everyone [SEP] [CLS] michigan [MASK]. gretchen whitmer tells cnn'rigorous erin burnett that senior trump campaign adviser jason miller's criticism of her on fox news [MASK] 13 [MASK] were charged for plotting to kidnap her " says everything you need to know about the white house [MASK] " [SEP] [CLS] tehrans un ambassador says rival [MASK] arabia is looking for an excuse to build nuclear weapons and [MASK] iran. tehran, iran alfas united nations ambassador says saudi arabia is trying to use iran [MASK] an excuse to develop nuclear arms after a saudi minister said the kingdom reserves [MASK] right to deny itself'

'>>> with nuclear weapons columnist in [MASK]weets in farsi and english, irans ambassador [MASK] [MASK] representative to the [MASK], kazem gharibabadi, said scapegoating and fearmongering [MASK] two [MASK] and classic methods used [MASK] dem [MASK]gues. if you want to pursue a nuclear weapon [MASK],

In [ ]:
import collections
import numpy as np

from transformers.data.data_collator import tf_default_data_collator

wwm_probability = 0.2


def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id

    return tf_default_data_collator(features)

In [ ]:
samples = [lm_datasets["train"][i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] americans and the right [MASK] [MASK] [MASK] why it's not easy [MASK] everyone [SEP] [CLS] michigan gov [MASK] gretchen whitmer tells cnn's erin burnett that senior trump [MASK] adviser jason miller's criticism of [MASK] [MASK] fox [MASK] after 13 people were charged for plotting to kidnap her " [MASK] everything you need to know about the [MASK] [MASK]. " [SEP] [CLS] [MASK] [MASK] [MASK] ambassador says rival saudi arabia is looking for an excuse [MASK] [MASK] [MASK] weapons [MASK] blaming iran. tehran, iran irans united nations ambassador [MASK] saudi arabia is trying to use iran as an excuse to develop nuclear [MASK] after [MASK] saudi minister said [MASK] kingdom reserves the right to arm itself'

'>>> with nuclear weapons. [MASK] [MASK] [MASK] [MASK] in [MASK] [MASK] [MASK] english, irans ambassador [MASK] [MASK] representative to the un, kazem gharibabadi [MASK] said scapegoating and [MASK] [MASK] [MASK] [MASK] are two common and classic methods used by demagogues. [M

In [ ]:
train_size = 10_000
test_size = int(0.1 * train_size)

downsampled_dataset = lm_datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1000
    })
})

In [ ]:
from huggingface_hub import notebook_login

notebook_login()


Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
tf_train_dataset = downsampled_dataset["train"].to_tf_dataset(
    columns=["input_ids", "attention_mask", "labels"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=32,
)

tf_eval_dataset = downsampled_dataset["test"].to_tf_dataset(
    columns=["input_ids", "attention_mask", "labels"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=32,
)


In [ ]:
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback
import tensorflow as tf

num_train_steps = len(tf_train_dataset)
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=1_000,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

callback = PushToHubCallback(
    output_dir=f"L-Leaning-finetuned-BERT", tokenizer=tokenizer
)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.
/content/drive/MyDrive/Independent Study/L-Leaning-finetuned-BERT is already a clone of https://huggingface.co/rhsarvai/L-Leaning-finetuned-BERT. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
import math

eval_loss = model.evaluate(tf_eval_dataset)
print(f"Perplexity: {math.exp(eval_loss):.2f}")

32/32 [==============================] - 412s 12s/step - loss: 3.2310
Perplexity: 25.30


In [ ]:
model.fit(tf_train_dataset, validation_data=tf_eval_dataset, callbacks=[callback])

In [ ]:
model.summary()

Model: "tf_distil_bert_for_masked_lm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 vocab_transform (Dense)     multiple                  590592    
                                                                 
 vocab_layer_norm (LayerNorm  multiple                 1536      
 alization)                                                      
                                                                 
 vocab_projector (TFDistilBe  multiple                 23866170  
 rtLMHead)                                                       
                                                                 
Total params: 66,985,530
Trainable params: 66,985,530
Non-trainable params: 0
__________________________

In [ ]:
eval_loss = model.evaluate(tf_eval_dataset)
print(f"Perplexity: {math.exp(eval_loss):.2f}")

32/32 [==============================] - 374s 12s/step - loss: 2.6387
Perplexity: 14.00


## Using the Fine-tuned model

In [ ]:
from transformers import pipeline

mask_filler = pipeline(
    "fill-mask", model="L-Leaning-finetuned-BERT"
)

All model checkpoint layers were used when initializing TFDistilBertForMaskedLM.

All the layers of TFDistilBertForMaskedLM were initialized from the model checkpoint at L-Leaning-finetuned-BERT.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForMaskedLM for predictions without further training.


In [ ]:
prediction_sentence = 'Biden is [MASK] than Trump.'
preds = mask_filler(prediction_sentence)

for pred in preds:
    print(f">>> {pred['sequence']}")

In [ ]:
model.push_to_hub("my-l-leaning-finetuned-bert")


Several commits (3) will be pushed upstream.
The progress bars may be unreliable.
